# Simple CLI chat loop that sends user queries to the GIS agent and prints the results


In [1]:
from core.planner import process_user_input
from core.sql_builder import plan_to_sql
from mcp_c_s.mcp_client import query_postgis 
from ui.map_utils import showMap

In [6]:
# async def chat_loop():
#     print("GIS agent chat – type 'quit' to stop.\n")

#     while True:
#         user_q = input("You: ").strip()
#         print("'''''''''''''''''''''''''''''''''''''''''''")

#         if user_q.lower() in ("quit", "exit", "q"):
#             print("Welcome back!")
#             break

#         plan = process_user_input(user_q)
#         if not isinstance(plan, dict):
#             continue

#         sql = plan_to_sql(plan)

#         df = await query_postgis(sql)  

#         if df is None or df.empty:
#             print("No results found.")
#             continue

#         display(df.head(10))
#         display(showMap(df))
# await chat_loop()


In [2]:
async def chat_once(user_q):
    plan = process_user_input(user_q)
    #plan = enrich_plan(plan)
    if not isinstance(plan, dict):
        print(plan)
        return

    sql = plan_to_sql(plan)
    df = await query_postgis(sql)

    if df is None or df.empty:
        print("No results.")
        return

    display(showMap(df))


In [6]:
await chat_once("5 buldings in drammen")
